Any time series may be split into the following components: Base Level + Trend + Seasonality + Error

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from random import gauss as gs
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from matplotlib.pylab import rcParams
%matplotlib inline

In [ ]:
fb = pd.read_csv('data/google-trends_football_us.csv').iloc[1:, :]
fb.columns = ['counts']

In [ ]:
fb['counts'] = fb['counts'].replace('<1', '0')
fb['counts'] = fb['counts'].astype(int)

In [ ]:
fb.tail()

## Series as Both Predictor and Target?

Often, the phenomenon we want to capture with a time series is a dataset being correlated with *itself*.

Well, of course every dataset is perfectly correlated with itself. But what we're after now is the idea that a series is correlated with *earlier versions* of itself.

Consider the problem of trying to predict tomorrow's closing price for some stock on the market. One may consider lots of features, like what sort of company it is to which the stock belongs or whether that company has been in the news recently.

But it is very often the case that one of the most helpful predictors of tomorrow's price is *today's* price. And so we want to build a model where one of our predictors is an earlier version of our target.

One tool we can use is **`df.rolling()`**, which creates a Rolling object that we can use to calculate statistics dynamically.

In [ ]:
fb.rolling(window=3).mean().head()

In [ ]:
fb['roll_avg'] = fb.rolling(window=2).mean()

fb.corr()

In [ ]:
plt.scatter(fb.index[:30], fb['counts'][:30])
plt.scatter(fb.index[1:31], fb['roll_avg'][1:31]);

In [ ]:
lr = LinearRegression()

lr.fit(fb[['roll_avg']][1:], fb['counts'][1:])

In [ ]:
plt.figure(figsize=(20, 4))
plt.plot(fb.index[1:25], fb['counts'][1:25], label='Data')
plt.plot(fb.index[1:25], lr.predict(fb[['roll_avg']][1:25]),
         label='Predicted')
plt.legend();

## Autocorrelation and Partial Autocorrelation
>We can calculate the correlation for time series observations with observations with previous time steps, called lags. Because the correlation of the time series observations is calculated with values of the same series at previous times, this is called a serial correlation, or an autocorrelation.

>The basic idea of autocorrelation is simple: See how a series correlates with a "lagged" version of itself. If my sequence is $S_0 = (x_0, x_1, x_2, ... , x_n)$, then I can measure the Pearson correlation between the first $n-k + 1$ terms of $S_0$ and $S_{lag} = (x_k, x_{k+1}, x_{k+2}, ... , x_n)$.


In [ ]:
acf(fb['counts'], nlags=20, fft=False)

In [ ]:
# Plot using `.autocorr()`

X = np.arange(0, 100, 1)
Y = np.zeros(100)
for j in X[1:]:
    Y[j] = fb['counts'].autocorr(lag=j)
plt.figure(figsize=(20, 4))
plt.plot(X, Y)
plt.grid();

In [ ]:
# To construct the autocorrelation function, we take the covariance of our time series with a lagged version
# and then divide by the variance of the series.

X = np.arange(0, 100, 1)
Y = np.zeros(100)
for j in X[1:]:
    Y[j] += np.cov(fb['counts'][:-j], fb['counts'][j:])[0, 1] / np.var(fb['counts']) * ((180-j) / 180)
plt.figure(figsize=(20, 4))
plt.plot(X, Y)
plt.grid();

In [ ]:
plt.figure(figsize=(20, 4))
pd.plotting.autocorrelation_plot(fb['counts']);

The horizontal bands represent condfidence intervals, which are calculated by taking relevant z-scores of the standard normal distribution and dividing by the square root of the number of observations. What do these intervals mean? 

In [ ]:
#We can also use the plot_acf() function from statsmodels:
rcParams['figure.figsize'] = 20, 4

plot_acf(fb['counts'], lags=125, alpha=None);

### Partial Autocorrelation
The idea behind partial Autocorrelation is to compare a series to a lagged version of itself while abstracting away from intermediate values. In effect, this amounts to exploring the correlations among residuals

In [ ]:
pacf(fb['counts'], nlags=20)

A common way of computing the partial autocorrelation is by fitting regressions to residuals from a simple dummy model that always predicts the mean. The coefficient of the final term will be the partial autocorrelation for the corresponding number of lags.

In [ ]:
y_tilde = fb['counts'] - fb['counts'].mean()

In [ ]:
x_1 = (fb['counts'][:-1] - fb['counts'].mean()).values.reshape(-1, 1)
x_2 = (fb['counts'][:-2] - fb['counts'].mean()).values.reshape(-1, 1)

In [ ]:
lr = LinearRegression()

lr.fit(np.concatenate([x_1[1:], x_2], axis=1), y_tilde[2:]).coef_[-1]

In [ ]:
x_1 = (fb['counts'][:-1] - fb['counts'].mean()).values.reshape(-1, 1)
x_2 = (fb['counts'][:-2] - fb['counts'].mean()).values.reshape(-1, 1)
x_3 = (fb['counts'][:-3] - fb['counts'].mean()).values.reshape(-1, 1)

In [ ]:
lr2 = LinearRegression()

lr2.fit(np.concatenate([x_1[2:], x_2[1:], x_3], axis=1), y_tilde[3:]).coef_[-1]

## ARMA Modeling 
'AR' is for "Auto-Regressive": The prediction for today will be a function of the value for previous days.

The number of lag periods we want to include will be a parameter in the statsmodels model object ("p").

In particular, auto-regressive models look like this:

$X_t = \beta_0 + \Sigma^p_{i=1}\beta_iX_{t-i} + \epsilon_t$,
where $\epsilon_t$ should be more or less accurately modeled by white noise.

We indicate how many terms our $AR$ model has by writing $AR(k)$ where $k$ is the number of terms.

Looking at the PACF can help us decide on an appropriate $p$: We can look at where the correlation values cross the confidence thresholds.


'MA' is for "Moving Average": The prediction for today will be a function of the rolling mean.

The number of average terms we want to include will be a parameter in the statsmodels model object ("q").

In particular, moving-average models look like this:

$X_t = \mu + \epsilon_t + \Sigma^q_{i=1}\beta_i\epsilon_{t-i}$,
where again the $\epsilon$ should be modeled by white noise.

We indicate how many terms our $MA$ model has by writing $MA(k)$ where $k$ is the number of terms.

Looking at the ACF can help us decide on an appropriate $q$: We can look at where the correlation values cross the confidence thresholds.
The $AR$ and $MA$ models are intimately related. In fact $AR(p)$ is equivalent to $MA(\infty)$ for any $p$. The reverse holds as well if $|\theta| < 1$ for all $\theta$ in $MA(q)$. For more on this, see [here](https://otexts.com/fpp2/MA.html).

Consider $AR(1)$:

$X_t = \beta_0 + \beta_1X_{t-1} + \epsilon_t$ <br/> 
$= \beta_0 + \beta_1(\beta_1X_{t-2} + \epsilon_{t-1})$ <br/>
$= \beta_0 + \beta_1^2X_{t-2} + \beta_1\epsilon_{t-1}$ <br/>
$= \beta_0 + \beta_1^3X_{t-3} + \beta_1^2\epsilon_{t-2} + \beta_1\epsilon_{t-1}$

In the limit of this expansion we obtain an expression for $MA(\infty)$.

## Stationarity and the Dickey-Fuller Test

ARMA models assume that the time series is stationary, which means that its statistical properties are not a (meaningful) function of time. That is, the statistical properties of the series like mean, variance and autocorrelation are constant over time. A stationary time series is devoid of seasonal effects as well.

It may seem counterintuitive that, for modeling purposes, we want our time series not to be a function of time! But the basic idea is that we want our datapoints to be mutually independent. Why? 

One way of testing for stationarity is to use the Dickey-Fuller Test. The statsmodels version returns the test statistic and a p-value, relative to the null hypothesis that the series in question is NOT stationary.

In [ ]:
# Presumably, our football series is not stationary. Let's check.
result = adfuller(fb['counts'], autolag=None)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


In [ ]:
#could try a log transform 
from numpy import log
X = log(fb['counts'])
result = adfuller(X, autolag=None)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
#rolling mean 
roll_mean = fb['counts'].rolling(window=4).mean()
fig = plt.figure(figsize=(11,7))
plt.plot(fb['counts'], color='blue',label='Original')
plt.plot(roll_mean, color='red', label='Rolling Mean')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

In [ ]:
# Subtract the moving average from the original data
#we are taking the average of the last four values
#the rolling mean is not defined for the first three values.
data_minus_roll_mean = fb['counts'] - roll_mean
data_minus_roll_mean.head(15)

In [ ]:
# Drop the missing values from time series calculated above
data_minus_roll_mean.dropna(inplace=True)

In [ ]:
fig = plt.figure(figsize=(11,7))
plt.plot(data_minus_roll_mean, color='blue',label='Sales - rolling mean')
plt.legend(loc='best')
plt.title('Sales while the rolling mean is subtracted')
plt.show(block=False)

In [ ]:
#differencing
data_diff = fb['counts'].diff(periods=1)
data_diff.head(10)

In [ ]:
fig = plt.figure(figsize=(11,7))
plt.plot(data_diff, color='blue',label='Sales - rolling mean')
plt.legend(loc='best')
plt.title('Differenced sales series')
plt.show(block=False)

## ARMA Modeling 

In [ ]:
# Instantiate & fit model 
p = 3
q = 1

# This model will have three auto-regressive terms and one moving-average term.

ar = ARMA(fb['counts'].diff().values[1:], (p, q)).fit()

In [ ]:
ar.summary()

In [ ]:
r2_score(fb['counts'].diff().values[1:], ar.predict())